# Python Decorators

Decorators are a powerful and useful tool in Python that allows you to modify the behavior of a function or class method. 

## 1. Introduction

A decorator is a function that takes another function and extends its behavior without explicitly modifying it. 

In [11]:
def decorator_function(original_function):
    def wrapper_function(*args, **kwargs):
        # Code to execute before the original function
        result = original_function(*args, **kwargs)
        # Code to execute after the original function
        return result
    return wrapper_function

# Usage
@decorator_function
def display():
    print("Display function ran")

display()

Display function ran


## 2. Basic Function Decorators

Let's start with a simple example:

In [12]:
def simple_decorator(func):
    def wrapper():
        print("Something is happening before the function is called.")
        func()
        print("Something is happening after the function is called.")
    return wrapper

@simple_decorator
def say_hello():
    print("Hello!")

say_hello()

Something is happening before the function is called.
Hello!
Something is happening after the function is called.


## 3. Decorators with Arguments


Decorators can also accept arguments:

In [13]:
def repeat(num_times):
    def decorator_repeat(func):
        def wrapper(*args, **kwargs):
            for _ in range(num_times):
                result = func(*args, **kwargs)
            return result
        return wrapper
    return decorator_repeat

@repeat(num_times=3)
def greet(name):
    print(f"Hello {name}")

greet("World")

Hello World
Hello World
Hello World


## 4. Class Decorators

Decorators can also be applied to classes:

In [14]:
class CountCalls:
    def __init__(self, func):
        self.func = func
        self.num_calls = 0

    def __call__(self, *args, **kwargs):
        self.num_calls += 1
        print(f"Call {self.num_calls} of {self.func.__name__}")
        return self.func(*args, **kwargs)

@CountCalls
def say_hello():
    print("Hello!")

say_hello()
say_hello()


Call 1 of say_hello
Hello!
Call 2 of say_hello
Hello!


## 5. Decorators for Method Functions

Decorators can be used to modify methods in a class:

In [15]:
def method_decorator(func):
    def wrapper(self, *args, **kwargs):
        print(f"Before {func.__name__}")
        result = func(self, *args, **kwargs)
        print(f"After {func.__name__}")
        return result
    return wrapper

class MyClass:
    @method_decorator
    def display(self):
        print("Display method called")

obj = MyClass()
obj.display()

Before display
Display method called
After display


## 6. Chaining Decorators

Multiple decorators can be chained together:

In [16]:
def decorator_one(func):
    def wrapper(*args, **kwargs):
        print("Decorator One")
        return func(*args, **kwargs)
    return wrapper

def decorator_two(func):
    def wrapper(*args, **kwargs):
        print("Decorator Two")
        return func(*args, **kwargs)
    return wrapper

@decorator_one
@decorator_two
def say_hello():
    print("Hello!")

say_hello()

Decorator One
Decorator Two
Hello!


## 7. Using Decorators for Logging


A common use of decorators is to log function calls:

In [17]:
import logging

logging.basicConfig(level=logging.INFO)

def log_function_call(func):
    def wrapper(*args, **kwargs):
        logging.info(f"Running function: {func.__name__}")
        return func(*args, **kwargs)
    return wrapper

@log_function_call
def process_data(data):
    print(f"Processing {data}")

process_data("Sample Data")

INFO:root:Running function: process_data


Processing Sample Data


## 8. Memoization with Decorators

Memoization is an optimization technique that can be implemented using decorators:

In [20]:
def memoize(func):
    # This dictionary stores the results of previous 
    # function calls with their respective arguments as keys.
    cache = {}
    def wrapper(*args):
        # If the result for the given arguments (args) is already in the cache
        if args in cache:
            print(f"Cache hit for args {args}: {cache[args]}")
            return cache[args]
        # If the result is not in the cache, it calls the original function (func),
        result = func(*args)
        # Stores the result in the cache,
        cache[args] = result
        print(f"Cache set for args {args}: {cache[args]}")
        return result
    return wrapper

@memoize
def fibonacci(n):
    if n in (0, 1):
        return n
    return fibonacci(n - 1) + fibonacci(n - 2)

print(fibonacci(35))


Cache set for args (1,): 1
Cache set for args (0,): 0
Cache set for args (2,): 1
Cache hit for args (1,): 1
Cache set for args (3,): 2
Cache hit for args (2,): 1
Cache set for args (4,): 3
Cache hit for args (3,): 2
Cache set for args (5,): 5
Cache hit for args (4,): 3
Cache set for args (6,): 8
Cache hit for args (5,): 5
Cache set for args (7,): 13
Cache hit for args (6,): 8
Cache set for args (8,): 21
Cache hit for args (7,): 13
Cache set for args (9,): 34
Cache hit for args (8,): 21
Cache set for args (10,): 55
Cache hit for args (9,): 34
Cache set for args (11,): 89
Cache hit for args (10,): 55
Cache set for args (12,): 144
Cache hit for args (11,): 89
Cache set for args (13,): 233
Cache hit for args (12,): 144
Cache set for args (14,): 377
Cache hit for args (13,): 233
Cache set for args (15,): 610
Cache hit for args (14,): 377
Cache set for args (16,): 987
Cache hit for args (15,): 610
Cache set for args (17,): 1597
Cache hit for args (16,): 987
Cache set for args (18,): 2584
Cac

## 9. Access Control with Decorators

Decorators can be used to control access to certain functions:

In [19]:
def requires_permission(permission):
    def decorator(func):
        def wrapper(user, *args, **kwargs):
            if user.permission == permission:
                return func(user, *args, **kwargs)
            else:
                raise PermissionError("Permission denied")
        return wrapper
    return decorator

class User:
    def __init__(self, permission):
        self.permission = permission

@requires_permission('admin')
def view_admin_dashboard(user):
    print("Admin dashboard")

user = User('admin')
view_admin_dashboard(user)

user = User('guest')
view_admin_dashboard(user) 

Admin dashboard


PermissionError: Permission denied

## 10. Example: Rate Limiting Decorator

A practical example of a rate-limiting decorator:

In [ ]:
import time
from functools import wraps

def rate_limit(max_calls, period=1.0):
    min_interval = period / float(max_calls)
    def decorator(func):
        last_called = [0.0]
        @wraps(func)
        def wrapper(*args, **kwargs):
            elapsed = time.time() - last_called[0]
            left_to_wait = min_interval - elapsed
            if left_to_wait > 0:
                time.sleep(left_to_wait)
            last_called[0] = time.time()
            return func(*args, **kwargs)
        return wrapper
    return decorator

@rate_limit(2, 5)  # Limit to 2 calls every 5 seconds
def print_message(message):
    print(message)

for i in range(5):
    print_message(f"Message {i}")

Message 0
Message 1
Message 2
Message 3
Message 4
